In [1]:
import numpy as np
import pandas as pd
import os

In [40]:
Dtype="inD" #可调
n=19 #可调
recordingMeta_file_path = "/DATA1/rzhou/ika/%(Dtype)s/data/%(n)02d_recordingMeta.csv" %{'Dtype':Dtype,'n':n}
df = pd.read_csv(recordingMeta_file_path)
num_frames_recording=df["frameRate"].iloc[0]*df["duration"].iloc[0]
numTracks=df["numTracks"].iloc[0]
print(num_frames_recording)
print(numTracks)

24836.0
370


In [41]:
import pandas as pd
import numpy as np
import math

# 参数设置
seq_length = 1000  # 每个案例的帧数长度
interval = 100  # 案例之间的开始帧数间隔

num_cases = math.ceil((num_frames_recording - seq_length) / interval) + 1

# 初始化列表存储每个案例的track IDs
trackIDLists = [[] for _ in range(num_cases)]

# 读取tracksMeta文件
csv_file_path = "/DATA1/rzhou/ika/%(Dtype)s/data/%(n)02d_tracksMeta.csv" % {'Dtype': Dtype, 'n': n}
df_tracksMeta = pd.read_csv(csv_file_path)

# Initialize an array to hold the pointer for each trackId
pointers = np.zeros(numTracks, dtype=int)
pointers[1:] = np.cumsum(df_tracksMeta['numFrames'].values)[:-1]

# 遍历每个track，确定它出现在哪些案例中
for index, row in df_tracksMeta.iterrows():
    initialFrame = row['initialFrame']
    finalFrame = row['finalFrame']
    numFrames = row['numFrames']

    # 跳过帧数过多的tracks
    if numFrames > 1500:
        continue

    # 确定track出现在哪些案例中
    for i in range(num_cases):
        case_start_frame = i * interval
        case_end_frame = case_start_frame + seq_length - 1
        # 检查track的帧区间是否与当前案例的帧区间有交集
        if not (finalFrame < case_start_frame or initialFrame > case_end_frame):
            trackIDLists[i].append(row['trackId'])

# 筛选出非空的trackIDLists
filtered_trackIDLists = [trackIDs for trackIDs in trackIDLists if trackIDs]

# 打印结果
for i, trackIDs in enumerate(filtered_trackIDLists):
    print(f"Case {i+1}: Track IDs: {trackIDs}")
# print(filtered_trackIDLists[32])

# 初始化一个列表来存储每个案例的track数量，并计算最多的track数量
track_counts = [len(track_ids) for track_ids in filtered_trackIDLists]
max_tracks = max(track_counts)

print(f"最多的track数量为: {max_tracks}")


Case 1: Track IDs: [0, 1, 2, 16, 17, 18, 19]
Case 2: Track IDs: [0, 2, 16, 17, 18, 19, 20, 21]
Case 3: Track IDs: [2, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Case 4: Track IDs: [2, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
Case 5: Track IDs: [16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
Case 6: Track IDs: [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
Case 7: Track IDs: [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Case 8: Track IDs: [17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
Case 9: Track IDs: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
Case 10: Track IDs: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
Case 11: Track IDs: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
Case 12: Track IDs: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
Case 13: Track IDs: [19, 20, 21, 22, 23,

In [29]:
import pandas as pd
import numpy as np
import math

# Parameters
seq_length = 1000  # Length of each case in frames
interval = 100  # Interval between the starts of cases

# Calculate the number of cases
num_cases = math.floor((num_frames_recording - seq_length) / interval) + 1
print(num_cases)

# Initialize the list to store track IDs for each case
trackIDLists = [list() for _ in range(num_cases)]

# Load the CSV file
csv_file_path = "/DATA1/rzhou/ika/%(Dtype)s/data/%(n)02d_tracksMeta.csv" %{'Dtype':Dtype,'n':n}
df_tracksMeta  = pd.read_csv(csv_file_path)
# Initialize an array to hold the pointer for each trackId
pointers = np.zeros(numTracks, dtype=int)
pointers[1:] = np.cumsum(df_tracksMeta['numFrames'].values)[:-1]
# Iterate through each track
for index, row in df_tracksMeta .iterrows():
    initialFrame = row["initialFrame"]
    finalFrame = row["finalFrame"]
    numFrames = row["numFrames"]
    
    # Skip tracks with too many frames (optional criterion)
    if numFrames > 1500:
        continue
    
    # Skip tracks if it is not car (optional criterion)
    # if row["class"] !="car":
    #     continue

    # Calculate the start and end case indexes for the current vehicle
    start_case_idx = max(0, math.ceil((initialFrame - seq_length) / interval))
    end_case_idx = min(num_cases - 1, math.floor(finalFrame / interval))
    
    # Add the track ID to the relevant cases
    for i in range(start_case_idx, end_case_idx + 1):
        trackIDLists[i].append(row["trackId"])

#Print the result
for i, trackIDs in enumerate(trackIDLists):
    print(f"Case {i+1}: Track IDs: {trackIDs}")
    
#print(trackIDLists[32])
    
# Filter out empty lists from trackIDLists
filtered_trackIDLists = [trackIDs for trackIDs in trackIDLists if trackIDs]
num_case=len(filtered_trackIDLists)

# 初始化一个列表来存储每个case的track数量
track_counts = [len(track_ids) for track_ids in filtered_trackIDLists]
print(f"track数量为: {track_counts}")

# 计算最多的track数量
max_tracks = max(track_counts)

print(f"最多的track数量为: {max_tracks}")
print(f"case数量为: {num_case}")


239
Case 1: Track IDs: [0, 1, 2, 16, 17, 18, 19]
Case 2: Track IDs: [0, 2, 16, 17, 18, 19, 20, 21]
Case 3: Track IDs: [2, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Case 4: Track IDs: [2, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
Case 5: Track IDs: [16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
Case 6: Track IDs: [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
Case 7: Track IDs: [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Case 8: Track IDs: [17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
Case 9: Track IDs: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
Case 10: Track IDs: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
Case 11: Track IDs: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
Case 12: Track IDs: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
Case 13: Track IDs: [19, 20, 21, 22,

In [42]:
pointers

array([     0,    106,    172,    514,  25350,  50186,  75022,  99858,
       124694, 149530, 174366, 199202, 224038, 248874, 273710, 298546,
       323382, 323585, 323988, 324271, 324981, 325435, 325595, 325788,
       326006, 326212, 326412, 326459, 326718, 326927, 327406, 327757,
       328485, 328811, 329837, 330636, 331698, 332596, 333574, 333766,
       334309, 334792, 335502, 335932, 336208, 336863, 337802, 338337,
       339226, 339527, 339952, 340442, 340693, 340974, 342341, 342736,
       343186, 343735, 344104, 344432, 345039, 345655, 345878, 346127,
       346475, 346732, 347063, 347459, 347764, 348614, 349932, 350497,
       351093, 351758, 351993, 352451, 353472, 354182, 354458, 355185,
       355587, 356111, 356404, 356660, 356933, 357239, 357499, 358537,
       358818, 359712, 359976, 360613, 360968, 361203, 361560, 362447,
       362749, 363641, 363897, 364138, 364460, 364945, 365259, 365597,
       366144, 366488, 366805, 366983, 367224, 367612, 368237, 368688,
      

In [8]:
# 创建列名
columns = ['caseID'] + [f'{xy}{i+1}' for i in range(max_tracks) for xy in ['x', 'y']]
# 计算总行数
total_rows = seq_length * num_cases
# 创建一个全是0的DataFrame
df = pd.DataFrame(0, index=range(total_rows), columns=columns)
print(df)
# 准备track数据
tracks_df = pd.read_csv("/DATA1/rzhou/ika/%(Dtype)s/data/%(n)02d_tracks.csv" %{'Dtype':Dtype,'n':n})
for caseid in range(num_cases):
    startframe_cur_case=interval*caseid
    #endframe_cur_case=startframe_cur_case+seq_length-1
    start_row=caseid*seq_length
    #end_row=start_row+seq_length-1
    for trackID_cur_case in range(len(filtered_trackIDLists[caseid])):
        trackID = filtered_trackIDLists[caseid][trackID_cur_case]
        initialFrame_cur_track=df_tracksMeta[trackID]["initialFrame"]
        finalFrame_cur_track=df_tracksMeta[trackID]["finalFrame"]
        start_frame_curcase_curtrack=max(startframe_cur_case,initialFrame_cur_track)
        end_frame_curcase_curtrack=min(endframe_cur_case,finalFrame_cur_track)
        start_row_in_df=start_row+start_frame_curcase_curtrack-startframe_cur_case
        end_row_in_df=start_row+end_frame_curcase_curtrack-startframe_cur_case
        start_row_in_tracksFile=pointers[trackID_cur_case]+start_frame_curcase_curtrack-initialFrame_cur_track
        #end_row_in_tracksFile=pointers[trackID_cur_case]+end_frame_curcase_curtrack-initialFrame_cur_track
        for r in range(start_row_in_df, end_row_in_df+1):
            df[r][x'trackID_cur_case+1']=tracks_df[start_row_in_tracksFile+r-start_row_in_df]["xCenter"]
            df[r][y'trackID_cur_case+1']=tracks_df[start_row_in_tracksFile+r-start_row_in_df]["yCenter"]
            
            
# Set caseID for each row
df['caseID'] = np.repeat(np.arange(1, num_cases + 1), seq_length)
        
        



        caseID  x1  y1  x2  y2  x3  y3  x4  y4  x5  ...  x30  y30  x31  y31  \
0            0   0   0   0   0   0   0   0   0   0  ...    0    0    0    0   
1            0   0   0   0   0   0   0   0   0   0  ...    0    0    0    0   
2            0   0   0   0   0   0   0   0   0   0  ...    0    0    0    0   
3            0   0   0   0   0   0   0   0   0   0  ...    0    0    0    0   
4            0   0   0   0   0   0   0   0   0   0  ...    0    0    0    0   
...        ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ...  ...   
238995       0   0   0   0   0   0   0   0   0   0  ...    0    0    0    0   
238996       0   0   0   0   0   0   0   0   0   0  ...    0    0    0    0   
238997       0   0   0   0   0   0   0   0   0   0  ...    0    0    0    0   
238998       0   0   0   0   0   0   0   0   0   0  ...    0    0    0    0   
238999       0   0   0   0   0   0   0   0   0   0  ...    0    0    0    0   

        x32  y32  x33  y33  x34  y34  
0         0 

In [43]:
# 创建列名
columns = ['caseID'] + [f'{xy}{i+1}' for i in range(max_tracks) for xy in ['x', 'y']]

# 计算总行数
total_rows = seq_length * num_cases

# 创建一个全是0的DataFrame，指定float64类型
df = pd.DataFrame(0, index=np.arange(total_rows), columns=columns).astype({'caseID': 'int'}).astype({col: 'float64' for col in columns if col != 'caseID'})

# 设置caseID列
df['caseID'] = np.repeat(np.arange(1, num_cases + 1), seq_length)

# 遍历每个案例
for case_id in range(num_cases):
    start_frame_of_case = interval * case_id
    start_row_of_case = case_id * seq_length

    # 遍历当前案例中的轨迹ID
    # for track_idx in range(len(filtered_trackIDLists[case_id])):
    #     track_id = filtered_trackIDLists[case_id][track_idx]
    for track_idx, track_id in enumerate(filtered_trackIDLists[case_id]):
        #print(track_idx)
        #print(track_id)
        # 获取轨迹的初始帧和最终帧
        initial_frame = df_tracksMeta.loc[track_id, "initialFrame"]
        final_frame = df_tracksMeta.loc[track_id, "finalFrame"]
        
        # 确定当前案例中轨迹的起止帧
        start_frame = max(start_frame_of_case, initial_frame)
        end_frame = min(start_frame_of_case + seq_length - 1, final_frame)
        
        # 计算在df中对应的起止行
        start_row = start_row_of_case + (start_frame - start_frame_of_case)
        end_row = start_row_of_case + (end_frame - start_frame_of_case)
        
        # 从tracks文件中提取轨迹数据
        track_rows_start = pointers[track_id] + (start_frame - initial_frame)
        track_rows_end = track_rows_start + (end_frame - start_frame + 1)
        #track_rows_end=pointers[track_id]+end_frame-initial_frame
        
        track_data = tracks_df.iloc[track_rows_start:track_rows_end]
        if track_data.empty:
            print(case_id)
            print(track_id)
            print("没有找到对应的track数据")
        
        # 更新DataFrame
        df.loc[start_row:end_row, f'x{track_idx+1}'] = track_data["xCenter"].values
        df.loc[start_row:end_row, f'y{track_idx+1}'] = track_data["yCenter"].values

# 注意：这里假设tracks_df是按trackId和frame排序的，而且pointers数组已正确初始化

num_features=2*max_tracks

output_file_path = "/DATA1/rzhou/ika/multi_testcases/%(Dtype)s/test/%(Dtype)s_map%(n)02d_interval%(interval)d_seq%(seq_length)d_nfea%(num_features)d.csv" %{'Dtype':Dtype,'n':n, 'interval': interval, 'seq_length': seq_length, 'num_features': num_features}
# 清空文件
with open(output_file_path, 'w') as file:
    pass

# 将DataFrame写入CSV
df.to_csv("/DATA1/rzhou/ika/multi_testcases/%(Dtype)s/test/%(Dtype)s_map%(n)02d_interval%(interval)d_seq%(seq_length)d_nfea%(num_features)d.csv" %{'Dtype':Dtype,'n':n, 'interval': interval, 'seq_length': seq_length, 'num_features': num_features}, index=False)

In [ ]:
# 创建空DataFrame
columns = ['caseID'] + [f'{xy}{i+1}' for i in range(max_tracks) for xy in ['x', 'y']]
df = pd.DataFrame(columns=columns)

# 准备track数据
tracks_df = pd.read_csv("/DATA1/rzhou/ika/%(Dtype)s/data/%(n)02d_tracks.csv" %{'Dtype':Dtype,'n':n})

rows_list = []
for case_idx in range(num_cases):
    print(f"case: {case_idx+1}")
    start_frame = case_idx * interval
    for frame in range(start_frame, start_frame + seq_length):
        row = {'caseID': case_idx + 1}  # caseID从1开始编号
        for track_id in trackIDLists[case_idx]:
            track_data = tracks_df[(tracks_df['trackId'] == track_id) & (tracks_df['frame'] == frame)]
            if not track_data.empty:
                x_center = track_data['xCenter'].values[0]
                y_center = track_data['yCenter'].values[0]
            else:
                x_center, y_center = 0, 0  # 如果在当前frame没有track数据，则填充0
            # 请注意这里直接更新row字典
            row[f'x{track_id}'] = x_center
            row[f'y{track_id}'] = y_center
        rows_list.append(row)

# 一次性将rows_list转换为DataFrame
new_rows_df = pd.DataFrame(rows_list)

# 现在可以将new_rows_df添加到原始df中
df = pd.concat([df, new_rows_df], ignore_index=True)

output_file_path = "/DATA1/rzhou/ika/multi_testcases/inD/test/%(Dtype)s_map%(n)02d_interval%(interval)d_seq%(seq_length)d_nfea%(num_features)d.csv" %{'Dtype':Dtype,'n':n, 'interval': interval, 'seq_length': seq_length, 'num_features': num_features}
# 清空文件
with open(output_file_path, 'w') as file:
    pass

# 将DataFrame写入CSV
df.to_csv("/DATA1/rzhou/ika/multi_testcases/inD/test/%(Dtype)s/%(Dtype)s_map%(n)02d_interval%(interval)d_seq%(seq_length)d_nfea%(num_features)d.csv" %{'Dtype':Dtype,'n':n, 'interval': interval, 'seq_length': seq_length, 'num_features': num_features}, index=False)

